In [85]:
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
import re
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from os import getcwd

In [2]:
# Download the twitter samples and stopwords from nltk

nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rajesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/rajesh/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [4]:
filepath = f"{getcwd()}/../tmp2/"

In [5]:
filepath

'/Users/rajesh/Documents/AI/NLP_Specialization/Tweet_Sentiment_Analysis_Naive_Bayes/../tmp2/'

In [6]:
print(nltk.data.path)

['/Users/rajesh/nltk_data', '/Users/rajesh/opt/anaconda3/envs/gen/nltk_data', '/Users/rajesh/opt/anaconda3/envs/gen/share/nltk_data', '/Users/rajesh/opt/anaconda3/envs/gen/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [7]:
nltk.data.path.append(filepath)

In [11]:
# get the sets of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_positive_tweets

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days',
 '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM',
 "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI",
 '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.',
 'Jgh , but we have to go to Bayan :D bye',
 'As an act of mischievousness, am calling the ETL layer of our in-house warehousing 

In [12]:
all_negative_tweets

['hopeless for tmr :(',
 "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(",
 '@Hegelbon That heart sliding into the waste basket. :(',
 '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
 'Dang starting next week I have "work" :(',
 "oh god, my babies' faces :( https://t.co/9fcwGvaki0",
 '@RileyMcDonough make me smile :((',
 '@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln',
 'why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2"',
 'Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz',
 "I have a really good m&amp;g idea but I'm never going to meet them :(((",
 '@Rampageinthebox mare ivan :(',
 '@SophiaMascardo happy trip, keep safe. see you soon :* :(',
 "I'm so tired hahahah :(",
 '@GrumpyCockney With knee replacements they get you up &amp; about the same day. :-(   Ou

In [13]:
print(len(all_positive_tweets))
print(len(all_negative_tweets))

5000
5000


In [16]:
# Split the data set for training and testing
train_pos = all_positive_tweets[:4000]
test_pos = all_positive_tweets[4000:]

train_neg = all_negative_tweets[:4000]
test_neg = all_negative_tweets[4000:]

In [17]:
print(len(train_pos),len(test_pos))

4000 1000


In [18]:
print(len(train_neg), len(test_neg))

4000 1000


In [19]:
train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [20]:
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [23]:
train_y

array([1., 1., 1., ..., 0., 0., 0.])

In [24]:
test_y

array([1., 1., 1., ..., 0., 0., 0.])

# Part 1: Process the Data

For any machine learning project, once you've gathered the data, the first step is to process it to make useful inputs to your model.
- **Remove noise**: You will first want to remove noise from your data -- that is, remove words that don't tell you much about the content. These include all common words like 'I, you, are, is, etc...' that would not give us enough information on the sentiment.
- We'll also remove stock market tickers, retweet symbols, hyperlinks, and hashtags because they can not tell you a lot of information on the sentiment.
- You also want to remove all the punctuation from a tweet. The reason for doing this is because we want to treat words with or without the punctuation as the same word, instead of treating "happy", "happy?", "happy!", "happy," and "happy." as different words.
- Finally you want to use stemming to only keep track of one variation of each word. In other words, we'll treat "motivation", "motivated", and "motivate" similarly by grouping them within the same stem of "motiv-".

We have given you the function `process_tweet()` that does this for you.

In [38]:
def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    # print(tweet_tokens)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [39]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# print cleaned tweet
print(process_tweet(custom_tweet))

['hello', 'there', '!', 'have', 'a', 'great', 'day', '.', ':)', 'good', 'morning']
['hello', 'great', 'day', ':)', 'good', 'morn']


## Part 1.1 Implementing your helper functions

To help train your naive bayes model, you will need to build a dictionary where the keys are a (word, label) tuple and the values are the corresponding frequency.  Note that the labels we'll use here are 1 for positive and 0 for negative.

You will also implement a `lookup()` helper function that takes in the `freqs` dictionary, a word, and a label (1 or 0) and returns the number of times that word and label tuple appears in the collection of tweets.

For example: given a list of tweets `["i am rather excited", "you are rather happy"]` and the label 1, the function will return a dictionary that contains the following key-value pairs:

{
    ("rather", 1): 2
    ("happi", 1) : 1
    ("excit", 1) : 1
}

- Notice how for each word in the given string, the same label 1 is assigned to each word.
- Notice how the words "i" and "am" are not saved, since it was removed by process_tweet because it is a stopword.
- Notice how the word "rather" appears twice in the list of tweets, and so its count value is 2.

#### Instructions
Create a function `count_tweets()` that takes a list of tweets as input, cleans all of them, and returns a dictionary.
- The key in the dictionary is a tuple containing the stemmed word and its class label, e.g. ("happi",1).
- The value the number of times this word appears in the given collection of tweets (an integer).

In [49]:
def count_tweets(result, tweets, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
    
    for y,tweet in zip(ys,tweets):
        for word in process_tweet(tweet):
            # define the key and which is label tuple
            pair = (word,y)
            
            # if the key exists in the dictionary , then increment the count
            if pair in result:
                result[pair] += 1
            # else the key is new, so add it to the dictionary and set the count to 1
            else:
                result[pair] = 1
                
    return result
            

In [50]:
# Testing your function
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

['i', 'am', 'happy']
['i', 'am', 'tricked']
['i', 'am', 'sad']
['i', 'am', 'tired']
['i', 'am', 'tired']


{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

# Part 2: Train your model using Naive Bayes

Naive bayes is an algorithm that could be used for sentiment analysis. It takes a short time to train and also has a short prediction time.

#### So how do you train a Naive Bayes classifier?
- The first part of training a naive bayes classifier is to identify the number of classes that you have.
- You will create a probability for each class.
$P(D_{pos})$ is the probability that the document is positive.
$P(D_{neg})$ is the probability that the document is negative.
Use the formulas as follows and store the values in a dictionary:

$$P(D_{pos}) = \frac{D_{pos}}{D}\tag{1}$$

$$P(D_{neg}) = \frac{D_{neg}}{D}\tag{2}$$

Where $D$ is the total number of documents, or tweets in this case, $D_{pos}$ is the total number of positive tweets and $D_{neg}$ is the total number of negative tweets.

#### Prior and Logprior

The prior probability represents the underlying probability in the target population that a tweet is positive versus negative.  In other words, if we had no specific information and blindly picked a tweet out of the population set, what is the probability that it will be positive versus that it will be negative? That is the "prior".

The prior is the ratio of the probabilities $\frac{P(D_{pos})}{P(D_{neg})}$.
We can take the log of the prior to rescale it, and we'll call this the logprior

$$\text{logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$$.

Note that $log(\frac{A}{B})$ is the same as $log(A) - log(B)$.  So the logprior can also be calculated as the difference between two logs:

$$\text{logprior} = \log (P(D_{pos})) - \log (P(D_{neg})) = \log (D_{pos}) - \log (D_{neg})\tag{3}$$

#### Positive and Negative Probability of a Word
To compute the positive probability and the negative probability for a specific word in the vocabulary, we'll use the following inputs:

- $freq_{pos}$ and $freq_{neg}$ are the frequencies of that specific word in the positive or negative class. In other words, the positive frequency of a word is the number of times the word is counted with the label of 1.
- $N_{pos}$ and $N_{neg}$ are the total number of positive and negative words for all documents (for all tweets), respectively.
- $V$ is the number of unique words in the entire set of documents, for all classes, whether positive or negative.

We'll use these to compute the positive and negative probability for a specific word using this formula:

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

Notice that we add the "+1" in the numerator for additive smoothing.  This [wiki article](https://en.wikipedia.org/wiki/Additive_smoothing) explains more about additive smoothing.

#### Log likelihood
To compute the loglikelihood of that very same word, we can implement the following equations:

$$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$$

##### Create `freqs` dictionary
- Given your `count_tweets()` function, you can compute a dictionary called `freqs` that contains all the frequencies.
- In this `freqs` dictionary, the key is the tuple (word, label)
- The value is the number of times it has appeared.

We will use this dictionary in several parts of this assignment.

In [51]:
# Build the freqs dictionary 
freqs = count_tweets({}, train_x, train_y)

['followfriday', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']
['hey', 'james', '!', 'how', 'odd', ':/', 'please', 'call', 'our', 'contact', 'centre', 'on', '02392441234', 'and', 'we', 'will', 'be', 'able', 'to', 'assist', 'you', ':)', 'many', 'thanks', '!']
['we', 'had', 'a', 'listen', 'last', 'night', ':)', 'as', 'you', 'bleed', 'is', 'an', 'amazing', 'track', '.', 'when', 'are', 'you', 'in', 'scotland', '?', '!']
['congrats', ':)']
['yeaaah', 'yipppy', '!', '!', '!', 'my', 'accnt', 'verified', 'rqst', 'has', 'succeed', 'got', 'a', 'blue', 'tick', 'mark', 'on', 'my', 'fb', 'profile', ':)', 'in', '15', 'days']
['this', 'one', 'is', 'irresistible', ':)', 'flipkartfashionfriday']
['we', "don't", 'like', 'to', 'keep', 'our', 'lovely', 'customers', 'waiting', 'for', 'long', '!', 'we', 'hope', 'you', 'enjoy', '!', 'happy', 'friday', '!', '-', 'lwwf', ':)']
['on', 'second', 'thought', ',', 'there', '’', 's', 'just', 'not', 'enough', 'time', 'for

['good', 'luck', 'to', 'lizaminnelli', 'for', 'her', 'upcoming', 'uk', 'appearances', ':-)']
['lol', '😄', '😄', 'really', '?', 'i', "can't", 'believe', 'a', 'beautiful', 'girl', 'like', 'you', 'are', 'single', ':p']
['i', 'will', 'never', 'be', 'listening', 'to', 'the', 'hills', 'every', 'again', ':-)']
['why', 'would', 'you', 'listen', 'to', 'beats', '1', '?', 'what', 'went', 'wrong', 'in', 'your', 'life', '?', ':)']
['haha', "that's", 'great', ':)']
['getting', 'everything', 'ready', 'for', 'my', 'natural', 'pefumery', 'workshop', 'at', 'neals', 'yard', ',', 'covent', 'garden', '-', 'tomorrow', ':)']
['fback', ':)', 'indo', 'harmos']
['just', 'americano', ':)']
['why', 'am', 'i', 'up', ':-)', ')']
['i', 'love', 'the', 'way', 'you', 'are', ':)', '<3']
['always', 'remember', ':)']
['aww', '!', 'thank', 'you', '.', 'beautiful', 'pic', ':D']
['heads', 'up', '!', ':)']
['hi', '!', 'saw', 'who', 'u', 'follow', 'and', 'thought', 'u', 'might', 'like', '"', 'dark', '"']
['smart', ',', 'handsho

['kill', ':)', 'me', ':)']
['how', 'are', 'those', 'pics', 'of', 'nash', 'coming', 'out', '?', ':)']
['i', 'make', 'sure', 'to', 'always', 'have', 'tissues', ',', 'chocolate', 'and', 'tea', 'for', 'every', 'hannibal', 'episode', ':)', 'it', 'just', "doesn't", 'work', 'any', 'other', 'way', 'for', 'me', ':)']
['contact', 'us', 'today', 'and', "we'll", 'get', 'you', 'out', 'of', 'the', 'city', 'into', 'a', "'", 'hotbed', "'", 'in', 'a', 'bush', 'somewhere', ':)', 'classicassures']
['hi', '!', 'would', 'you', 'like', 'an', 'concert', '?', 'let', 'me', 'know', 'your', 'city', 'and', 'country', 'and', "i'll", 'start', 'working', 'on', 'it', '!', 'thanks', ':)']
['stats', 'for', 'the', 'day', 'have', 'arrived', '.', '1', 'new', 'follower', 'and', 'no', 'unfollowers', ':)', 'via']
['glad', 'that', 'you', 'liked', '.', 'check', 'some', 'more', 'nice', 'stuff', 'at', 'wsalelove', ':)']
['follow', '&']
['we', 'are', 'thrilled', 'to', 'be', 'on', 'our', 'first', 'international', 'assignment', 'as

['sorted', ':)', '.', 'thanks', '.', 'daaru', 'party', 'in', 'my', 'chaddi', ',', 'bros', '.']
['beginning', 'of', 'line-up', ':)', 'it', 'is', 'left', 'for', 'the', 'line-up', '(', 'y', ')']
['i', "don't", 'know', 'how', 'you', 'feel', 'about', 'me', 'which', 'means', 'means', 'i', "don't", 'know', 'how', 'to', 'act', 'around', 'you', ':)', ')', '(', '(']
['ha', 'ha', ':-)', 'i', 'guess', 'i', 'was', 'being', 'polite', '.']
['thanks', 'so', 'much', ':-)']
['no', 'sleep', 'tx', ':)']
['looking', 'wonderful.great', 'body', '.', 'love', 'your', 'really', 'closely', 'trimmed', 'pubes', ':)', 'xx']
['es', 'mi', 'tio', ':)', 'you', 'know', 'its', 'a', 'must', 'to', 'follow', 'him', 'since', "i'm", 'from', 'sinaloa', '..', 'arre', '!']
['we', 'must', '!', '!', '!', 'hahaha', ':p']
['really', 'very', 'stylish', '&', 'trendy', '!', 'good', 'going', 'kim', ',', 'keep', 'it', 'up', ':)']
['love', 'it', '!', 'have', 'a', 'fabfriday', ':)']
['facetime', 'with', 'calum', 'on', 'the', 'weekend', 'wh

['first', 'driving', 'lesson', 'in', '10', 'months', '-', "let's", 'see', 'if', "i'm", 'still', 'shit', ':)']
['en', 'mis', 'manos', '.', 'holding', 'it', 'in', 'my', 'hands', ':)']
['true', 'that', ':)']
['when', 'the', 'managers', 'at', 'your', 'job', 'tell', 'you', 'and', 'your', 'friend', 'that', 'you', 'are', 'the', 'two', 'prettiest', 'girls', 'there', ':)', ':)', ':)', ':)', ':)', ':)', ':)']
['and', 'thank', 'you', 'to', 'wonderful', 'readers', 'like', 'you', ',', 'rose', ',', 'who', 'make', 'it', 'all', 'worthwhile', '.', 'have', 'a', 'lovely', 'weekend', '.', ':)']
['fback', '?', ':)']
['stats', 'for', 'the', 'day', 'have', 'arrived', '.', '1', 'new', 'follower', 'and', 'no', 'unfollowers', ':)', 'via']
['u', 'cant', 'change', 'how', 'people', 'feel', 'about', 'u', 'so', 'dnt', 'try', ',', 'just', 'live', 'ur', 'life', 'and', 'be', 'happy', ':)']
['he', 'said', "i'm", 'his', 'ideal', 'type', 'in', 'weekly', 'idol', 'and', 'make', 'a', 'love', 'pose', 'for', 'me', '>', '<', 'i

...

['so', 'true', ':(']
['done', 'can', 'u', 'please', 'give', 'them', 'to', '?', 'i', 'know', 'im', 'late', 'sorry', ':(']
[':(', '(', '(', 'thank', 'u', 'bby', '.', 'i', 'love', 'u', 'too', '.']
['can', 'i', 'just', 'call', 'into', 'work', ':(']
['last', 'day', ':-(']
['i', 'want', 'to', 'sleep', ':(']
['2nd', 'thoughts', 'on', 'college', ':(', '(']
['where', 'do', 'u', 'guys', 'stream', '?', 'my', 'stream', 'is', 'so', 'lq', 'im', '-', ':(']
['you', ',', 'with', 'your', 'words', 'like', 'knives', ':(', '(']
["can't", 'sleep', 'cause', "i've", 'had', 'the', 'worst', 'anxiety', 'all', 'day', ':(', 'ugh', 'can', 'i', 'just', 'stay', 'home', 'tomorrow', '?', '😞', 'sotired', 'mybrainneedstoshutoff']
['was', 'so', 'exited', 'to', 'get', 'a', 'maccies', 'breakfast', 'but', 'no', 'i', 'was', '10', 'minutes', 'late', 'and', 'had', 'to', 'get', 'a', 'burger', 'and', 'cheese', 'bites', ':(']
['it', 'was', 'amazing', '!', '!', 'such', 'a', 'short', 'trip', 'though', ':(', 'i', 'wanna', 'go', 'agai

['he', 'always', 'does', 'this', 'when', 'his', 'hair', 'is', 'getting', 'long', ':(']
['followed', 'me', 'thanks', ',', 'and', 'please', 'followed', 'me', 'too', ':(']
['fever', ':(', '(', '(']
['yeah', 'he', 'did', 'bc', 'at', 'the', 'end', 'he', 'run', 'back', 'to', 'his', 'place', 'hahaha', 'i', 'didnt', 'saw', 'them', 'laughing', 'tho', 'bc', 'low', 'quality', ':(']
['hi', 'danielle', ',', 'oh', 'no', ':(', 'do', 'you', 'have', 'the', 'latest', 'software', 'installed', '?', 'have', 'you', 'tried', 'doing', 'a', 'back', 'up', 'and', 'restore', '?']
['why', 'momo', ',', 'why', '?', ':(']
['why', 'you', 'unfollow', 'me', '?', ':(']
['like', 'pharma', ',', 'sugar', '...', 'almost', 'immovable', ':(']
['why', 'is', 'got', '7', "'", 's', 'outfit', 'for', 'music', 'bank', 'so', 'messy', '?', ':(', 'the', 'colour', 'could', 'have', 'been', 'better']
['thank', 'you', 'anshe', ':(', 'idk', 'i', 'just', "can't", 'bring', 'myself', 'to', 'get', 'it', 'off', 'my', 'mind', 'but', 'thank', 'you'

['why', 'have', 'a', 'no', 'been', 'paid', '?', '?', ':(', '(', '(']
['oh', 'no', '!', '!', 'it', 'wasnt', 'a', 'lunch', 'talk', '?', 'i', 'thought', 'it', 'started', 'at', '1pm', ':(']
['then', 'it', 'might', 'be', 'hard', 'to', 'get', 'into', 'final', 'fantasy', 'then', 'cause', 'the', 'newer', 'ones', 'are', 'pish', 'in', 'comparison', ':(', 'get', 'the', 'remastered', '10', 'for', 'ps4']
['i', 'keep', 'changing', 'between', 'fe14', 'icons', 'i', 'dont', 'know', 'which', 'one', 'to', 'pick', '>:(']
['.', 'yeah', 'doesn', '’', 't', 'appear', 'to', 'be', 'working', '.', 'but', 'i', 'want', 'a', 'free', 'cornetto', ':(', '(', 'not', 'a', 'strawberry', 'one', ',', 'they', '’', 're', 'a', 'fake', 'cornetto', ')']
['"', 'what', 'happened', '?', '"', '"', 'we', 'just', 'stopped', 'talking', '"', ':(']
['i', 'did', ':-(']
['please', 'bring', 'me', 'back', ':(']
['sorry', ':-(', '.', 'hope', 'it', 'gets', 'better']
['things', 'that', 'i', 'regret', 'until', 'now', '...', 'when', 'we', 'loose

['nawwwe', 'hang', 'in', 'there', 'love', ':(', 'yes', 'yes', 'chikka', 'ug', 'kita', 'rata', 'soonest', '!', ':', '*', 'mwamwa']
['i', "can't", 'get', 'it', 'to', 'fit', ':(']
['faggot', '*', ':(', 'poor', 'english', 'teacher']
['i', 'was', 'listening', 'on', 'radio', ':-(']
['yep', ':(', 'exactly', '.']
['id', 'do', 'anything', 'to', 'go', 'thorpe', 'park', 'tomorrow', '!', '!', '!', ':(']
['i', 'wanna', 'go', 'to', 'vidcon', ':(']
['such', 'an', 'eye', 'opener', ',', 'i', "didn't", 'realise', 'things', 'like', 'this', 'happened', '!', 'so', 'scary', 'and', 'bad', 'someone', 'would', 'do', 'that', ':(']
['can', 'someone', 'send', 'me', 'a', 'screenshot', 'of', 'this', 'conversation', 'i', 'want', 'to', 'see', 'what', 'it', 'was', 'but', 'my', 'phone', 'is', 'being', 'stupid', ':-(']
['i', 'doubt', "i'm", 'going', 'to', 'bed', 'soon', 'though', '.', ':(', 'i', 'wake', 'up', 'at', 'like', '9', '.']
['fyi', 'the', 'url', 'on', 'your', 'profile', "doesn't", 'work', ':(']
['but', "he's", 

['no', 'idk', 'if', 'i', 'wanna', 'watch', 'the', 'episode', 'now', ':(']
['why', 'my', 'lover', ':(', 'you', 'going', 'zoo', '?']
['baby', '>', 'boyfriend', '>', 'bitch', '?', '?', '?', 'i', "don't", 'know', 'how', 'to', 'complete', 'it', ':(']
['all', 'ice', 'cream', 'vans', 'are', 'busy', ':(', 'maybe', 'later', '...', 'ubericecream']
['hopefully', 'not', ':(', 'but', 'sulli', 'was', 'like', 'that', 'too', '...', 'well', ',', 'there', 'was', 'injured', 'amber', 'too', ',', 'but', "she's", 'still', 'there', 'haha', 'sigh', 'f', '(', 'x', ')']
['ouch', ':(', "what's", 'happened', 'to', 'make', 'you', 'say', 'this', 'darren', '?', 'is', 'there', 'anything', 'we', 'can', 'help', 'you', 'with', 'from', 'here', '?']
['not', 'even', 'sleepy', ':(']
['only', 'cpm', 'has', 'condemned', 'it', ',', 'rest', 'all', 'political', 'parties', 'are', 'quiet', ':(']
['when', 'you', 'realise', 'that', '2/3', 'of', 'your', 'big', 'summer', 'plans', 'have', 'been', 'and', 'gone', ':(', ':(', ':(', 'next'

In [54]:
freqs  # ('followfriday', 1.0): 23 --> 'followfriday' is the key, 1 means psoitive, 23 means how many times its there in corpus

{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

#### Instructions
Given a freqs dictionary, `train_x` (a list of tweets) and a `train_y` (a list of labels for each tweet), implement a naive bayes classifier.

##### Calculate $V$
- You can then compute the number of unique words that appear in the `freqs` dictionary to get your $V$ (you can use the `set` function).

##### Calculate $freq_{pos}$ and $freq_{neg}$
- Using your `freqs` dictionary, you can compute the positive and negative frequency of each word $freq_{pos}$ and $freq_{neg}$.

##### Calculate $N_{pos}$ and $N_{neg}$
- Using `freqs` dictionary, you can also compute the total number of positive words and total number of negative words $N_{pos}$ and $N_{neg}$.

##### Calculate $D$, $D_{pos}$, $D_{neg}$
- Using the `train_y` input list of labels, calculate the number of documents (tweets) $D$, as well as the number of positive documents (tweets) $D_{pos}$ and number of negative documents (tweets) $D_{neg}$.
- Calculate the probability that a document (tweet) is positive $P(D_{pos})$, and the probability that a document (tweet) is negative $P(D_{neg})$

##### Calculate the logprior
- the logprior is $log(D_{pos}) - log(D_{neg})$

##### Calculate log likelihood
- Finally, you can iterate over each word in the vocabulary, use your `lookup` function to get the positive frequencies, $freq_{pos}$, and the negative frequencies, $freq_{neg}$, for that specific word.
- Compute the positive probability of each word $P(W_{pos})$, negative probability of each word $P(W_{neg})$ using equations 4 & 5.

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

**Note:** We'll use a dictionary to store the log likelihoods for each word.  The key is the word, the value is the log likelihood of that word).

- You can then compute the loglikelihood: $log \left( \frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$.

In [56]:
print(freqs.keys())

dict_keys([('followfriday', 1.0), ('top', 1.0), ('engag', 1.0), ('member', 1.0), ('commun', 1.0), ('week', 1.0), (':)', 1.0), ('hey', 1.0), ('jame', 1.0), ('odd', 1.0), (':/', 1.0), ('pleas', 1.0), ('call', 1.0), ('contact', 1.0), ('centr', 1.0), ('02392441234', 1.0), ('abl', 1.0), ('assist', 1.0), ('mani', 1.0), ('thank', 1.0), ('listen', 1.0), ('last', 1.0), ('night', 1.0), ('bleed', 1.0), ('amaz', 1.0), ('track', 1.0), ('scotland', 1.0), ('congrat', 1.0), ('yeaaah', 1.0), ('yipppi', 1.0), ('accnt', 1.0), ('verifi', 1.0), ('rqst', 1.0), ('succeed', 1.0), ('got', 1.0), ('blue', 1.0), ('tick', 1.0), ('mark', 1.0), ('fb', 1.0), ('profil', 1.0), ('15', 1.0), ('day', 1.0), ('one', 1.0), ('irresist', 1.0), ('flipkartfashionfriday', 1.0), ('like', 1.0), ('keep', 1.0), ('love', 1.0), ('custom', 1.0), ('wait', 1.0), ('long', 1.0), ('hope', 1.0), ('enjoy', 1.0), ('happi', 1.0), ('friday', 1.0), ('lwwf', 1.0), ('second', 1.0), ('thought', 1.0), ('’', 1.0), ('enough', 1.0), ('time', 1.0), ('dd',

In [58]:
vocab = [pair[0] for pair in freqs.keys()]

In [59]:
print(len(vocab))

11338


In [61]:
vocab = set([pair[0] for pair in freqs.keys()])

In [71]:
vocab

{'briliant',
 'sumer',
 'deep',
 'toptravelcentar',
 'fenja',
 'fahrinahmad',
 "standen'",
 'crave',
 'slovak',
 'decemb',
 'organ',
 'comma',
 'crackl',
 'isi',
 'consum',
 '★',
 'zoom',
 'rubi',
 'mend',
 'monica',
 'lil',
 'nurs',
 'biblethump',
 'sexysasunday',
 'kayla',
 'shade',
 'teleport',
 'partner',
 'brooklyn',
 '0330 333 7234',
 'niwll',
 'sane',
 'swipe',
 "freddy'",
 'zayniscomingbackonjuli',
 'je',
 'arummzz',
 'dresscod',
 'tweenie_fox',
 'warfar',
 'cut',
 'ash',
 'ferdou',
 'climb',
 'teh',
 'singer',
 "o'neal",
 'reachabl',
 'cheshir',
 'els',
 'foreal',
 '11pm',
 'linda',
 'tsktsk',
 'emo',
 'christian',
 'borrow',
 'todayi',
 '10',
 'charl',
 'thur',
 'mj',
 'sooo',
 'nt4',
 'talk-kama',
 'netizen',
 'hiya',
 'vid',
 '☔',
 'disgust',
 'haaa',
 'histori',
 'softbal',
 'hatessuc',
 '6gb',
 'karibumombasa',
 'srsli',
 'confirm',
 'huski',
 'lsh',
 'coupl',
 'hawako',
 'nerd',
 'beginn',
 'excit',
 'viner',
 'amelia',
 'effect',
 'kong',
 'cute',
 'robert',
 'hound',
 

In [63]:
print(len(vocab)) # Gives unique words

9086


In [69]:
# Calculate N_pos, N_neg
N_pos = N_neg = 0
for pair in freqs.keys():
    if pair[1] > 0:
        N_pos += freqs[pair]
    else:
        N_neg += freqs[pair]

In [65]:
N_pos

26842

In [66]:
N_neg

27025

In [86]:
def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [87]:
a=0
for word in vocab:
    a += 1
    freq_pos = lookup(freqs, word, 1)
    freq_neg = lookup(freqs, word, 0)

In [88]:
a


9086

In [89]:
freq_neg

0

In [90]:
train_y

array([1., 1., 1., ..., 0., 0., 0.])

In [96]:
D_pos = list(filter(lambda x: x > 0, train_y))

In [97]:
D_pos

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [101]:
D_neg = list(filter(lambda x: x <= 0, train_y))

In [102]:
D_neg

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [113]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    loglikelihood = {}
    logprior = 0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos and N_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:

            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs[pair]

        # else, the label is negative
        else:

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
    D_pos = (len(list(filter(lambda x: x > 0, train_y))))

    # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)

    ### END CODE HERE ###
    #print(logprior)
    #print(loglikelihood)

    return logprior, loglikelihood

In [114]:
logprior, loglikelyhood = train_naive_bayes(freqs, train_x, train_y)

0.0
{'briliant': 0.6982277728402158, 'sumer': -0.6880665882796749, 'deep': 0.005080592280270417, 'toptravelcentar': 0.6982277728402158, 'fenja': 0.6982277728402158, 'fahrinahmad': 0.6982277728402158, "standen'": -0.6880665882796749, 'crave': -1.3812137688396202, 'slovak': -0.6880665882796749, 'decemb': 0.0050805922802706375, 'organ': -0.4003845158278938, 'comma': 0.6982277728402158, 'crackl': 0.005080592280270417, 'isi': 0.005080592280270417, 'consum': -0.6880665882796749, '★': 1.7968400615083255, 'zoom': -0.6880665882796749, 'rubi': -0.6880665882796749, 'mend': -0.6880665882796749, 'monica': 0.005080592280270417, 'lil': -0.46492303696546516, 'nurs': -0.6880665882796749, 'biblethump': -0.6880665882796749, 'sexysasunday': -0.4003845158278938, 'kayla': -0.6880665882796749, 'shade': -0.6880665882796749, 'teleport': 0.6982277728402158, 'partner': -1.0935316963878392, 'brooklyn': -0.4003845158278938, '0330 333 7234': 0.6982277728402158, 'niwll': -0.6880665882796749, 'sane': -0.6880665882796

In [115]:
print(logprior)

0.0


In [116]:
print(loglikelyhood)

{'briliant': 0.6982277728402158, 'sumer': -0.6880665882796749, 'deep': 0.005080592280270417, 'toptravelcentar': 0.6982277728402158, 'fenja': 0.6982277728402158, 'fahrinahmad': 0.6982277728402158, "standen'": -0.6880665882796749, 'crave': -1.3812137688396202, 'slovak': -0.6880665882796749, 'decemb': 0.0050805922802706375, 'organ': -0.4003845158278938, 'comma': 0.6982277728402158, 'crackl': 0.005080592280270417, 'isi': 0.005080592280270417, 'consum': -0.6880665882796749, '★': 1.7968400615083255, 'zoom': -0.6880665882796749, 'rubi': -0.6880665882796749, 'mend': -0.6880665882796749, 'monica': 0.005080592280270417, 'lil': -0.46492303696546516, 'nurs': -0.6880665882796749, 'biblethump': -0.6880665882796749, 'sexysasunday': -0.4003845158278938, 'kayla': -0.6880665882796749, 'shade': -0.6880665882796749, 'teleport': 0.6982277728402158, 'partner': -1.0935316963878392, 'brooklyn': -0.4003845158278938, '0330 333 7234': 0.6982277728402158, 'niwll': -0.6880665882796749, 'sane': -0.6880665882796749,

In [117]:
print(len(loglikelyhood))

9086


# Part 3: Test your naive bayes

Now that we have the `logprior` and `loglikelihood`, we can test the naive bayes function by making predicting on some tweets!

#### Implement `naive_bayes_predict`
**Instructions**:
Implement the `naive_bayes_predict` function to make predictions on tweets.
* The function takes in the `tweet`, `logprior`, `loglikelihood`.
* It returns the probability that the tweet belongs to the positive or negative class.
* For each tweet, sum up loglikelihoods of each word in the tweet.
* Also add the logprior to this sum to get the predicted sentiment of that tweet.

$$ p = logprior + \sum_i^N (loglikelihood_i)$$

#### Note
Note we calculate the prior from the training data, and that the training data is evenly split between positive and negative labels (4000 positive and 4000 negative tweets).  This means that the ratio of positive to negative 1, and the logprior is 0.

The value of 0.0 means that when we add the logprior to the log likelihood, we're just adding zero to the log likelihood.  However, please remember to include the logprior, because whenever the data is not perfectly balanced, the logprior will be a non-zero value.

In [118]:
def naive_bayes_predict(tweet, logprior, loglikelyhood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    # process the tweet to get the list of the words
    word_l = process_tweet(tweet)
    
    # initialize probability to zero
    p = 0
    
    # add the logprior
    p += logprior
    
    for word in word_l:
        
        # check if the word exists in the loglikelihood dictionary
        if word in loglikelyhood:
            # add the log likelyhood of that word to the probability 
            p += loglikelyhood[word]
            
    return p

In [119]:
# Experiment with your own tweet
my_tweet = 'She smiled.'
p = naive_bayes_predict(my_tweet, logprior, loglikelyhood)
print(f'The Expected output is : {p}')

['she', 'smiled', '.']
The Expected output is : 1.5736965101941158


**Expected Output**:
- The expected output is around 1.57
- The sentiment is positive.

In [121]:
# Experiment with your own tweet
my_tweet = 'I am disappointed by watching that movie'
p = naive_bayes_predict(my_tweet, logprior, loglikelyhood)
print(f'The Expected output is : {p}')

['i', 'am', 'disappointed', 'by', 'watching', 'that', 'movie']
The Expected output is : -2.3404531414671825


#### Implement test_naive_bayes
**Instructions**:
* Implement `test_naive_bayes` to check the accuracy of your predictions.
* The function takes in your `test_x`, `test_y`, log_prior, and loglikelihood
* It returns the accuracy of your model.
* First, use `naive_bayes_predict` function to make predictions for each tweet in text_x.

In [160]:
def test_naive_bayes(test_x, test_y, logprior, loglikelyhood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelyhood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = 1-error

    ### END CODE HERE ###
    print(accuracy)

    return accuracy

In [161]:
accuracy = test_naive_bayes(test_x, test_y, logprior, loglikelyhood)

['bro', ':', 'u', 'wan', 'cut', 'hair', 'anot', ',', 'ur', 'hair', 'long', 'liao', 'bo', 'me', ':', 'since', 'ord', 'liao', ',', 'take', 'it', 'easy', 'lor', 'treat', 'as', 'save', 'leave', 'it', 'longer', ':)', 'bro', ':', 'lol', 'sibei', 'xialan']
['is', 'back', '!', 'thnx', 'god', '!', '!', '!', "i'm", 'so', 'happy', ':)']
['thought', 'it', 'was', 'my', 'ears', 'which', 'were', 'malfunctioning', ',', 'thank', 'goodness', 'you', 'cleared', 'that', 'one', 'up', 'with', 'an', 'apology', ':-)']
["'", 'stuck', 'in', 'the', 'centre', 'right', 'with', 'you', '.', 'clowns', 'to', 'the', 'right', ',', 'jokers', 'to', 'the', 'left', '...', "'", ':)']
['happy', 'friday', ':-)']
['we', 'are', 'following', 'you', 'now', ':)', 'x']
['my', 'teenchoice', 'for', 'choiceinternationalartist', 'is', 'superjunior', 'fighting', 'oppa', ':D']
["it's", 'my', 'birthday', 'today', 'so', 'for', 'my', 'birthday', 'wish', 'i', 'hope', "there's", 'good', 'news', 'about', 'ben', 'soon', ':-)']
['good', 'morning',

['can', 'i', 'just', 'meet', 'harry', "that's", 'all', 'i', 'need', 'in', 'life', ':-)']
['stats', 'for', 'the', 'week', 'have', 'arrived', '.', '1', 'new', 'follower', 'and', 'no', 'unfollowers', ':)', 'via']
['one', 'more', 'sleep', 'til', 'the', 'wedding', ':-)']
['excited', 'about', 'the', 'chalkhill', 'park', 'celebration', 'tomorrow', 'from', 'midday', '.', 'come', 'join', 'the', 'fun', ':-)']
['hello', '!', '!', ':)', 'love', 'you', 'carter', ':)']
['what', 'a', 'good', 'read', ':)']
['you', 'too', ':-)']
['enjoy', ',', 'hope', 'weather', 'holds', 'for', 'you', 'guys', ':-)']
['done', ':)']
['the', 'remedies', 'the', 'media', "won't", 'tell', 'you', 'about', ':-)']
['thank', 'you', ':)']
['jumma', 'mubarak', 'all', 'of', 'you', '.', 'stay', 'blessed', ':)']
["we're", 'looking', 'forward', 'to', 'being', 'able', 'to', 'spend', 'more', 'time', 'with', 'you', '!', ':)', 'enjoy', 'iflix', '!', ':)']
['tgif', "it's", 'weekend', '...', 'enjoy', ':)']
['im', 'thankful', 'to', 'have', '

['has', 'a', 'got', 'out', 'of', 'bed', 'yet', '?', ':-)']
['awww', '!', 'sige', 'next', 'time', 'u', 'know', 'na', 'punta', 'kayo', 'dun', ',', 'imma', 'make', 'sure', 'na', "we'll", 'go', 'din', '!', ':-)']
['sure', 'thing', ':)', 'x']
['...', 'because', "it's", 'friday', ':D', '(', 'y', ')']
['of', 'course', ',', 'i', 'would', 'like', 'that', ':)', 'x']
['nooo', 'i', 'didnt', 'even', 'know', 'thats', 'how', 'you', 'actually', 'typed', 'it', 'out', 'i', 'googled', ':)']
['i', 'found', 'my', 'prompt', ':-)']
['just', 'chillin', 'w', '/', 'my', 'salmon', ':)']
['to', 'see', 'beauty', ',', 'you', 'have', 'to', 'believe', 'in', 'its', 'existence', 'first', ':)']
['hi', 'j', ',', 'you', 'can', 'upgrade', 'from', 't-mobile', ',', 'orange', 'or', 'ee', 'with', 'us', '.', 'dm', 'your', 'mobile', 'number', 'to', 'me', 'and', "i'll", 'ask', 'sales', 'to', 'call', 'you', ':)']
['congratulations', ',', 'really', 'looking', 'forward', 'to', 'the', 'book', 'and', 'some', 'good', 'sunday', 'morning

["i'm", 'omw', '😍', 'lol', 'i', 'wish', ':(', '(', '(']
['joke', 'lang', 'eh', ':(', 'hahdhdhshhs']
['never', 'seeing', 'your', 'dad', 'until', 'midnight', 'bc', 'he', 'worked', 'hard', 'as', 'fuck', ':(', 'growinguppoor']
['number', 'one', 'in', 'america', '!', '!', '😭', '🇺', '🇸', 'i', 'wish', 'i', 'could', 'buy', 'it', 'but', 'no', 'money', ':(']
['wish', 'that', 'i', 'can', 'buy', 'all', "bangtan's", 'merch', ':(']
['the', 'pain', 'of', 'all', 'ubericecream', 'drivers', 'being', 'busy', ':(']
['oh', 'gosh', 'what', 'did', 'you', 'say', '?', 'and', 'aw', 'hun', ':(', '*', 'cuddles', '*']
['me', 'wants', '!', ':(']
['taimoor', 'meray', 'dost', ':(']
['tyas', 'mind', 'to', 'refollow', 'me', ':(']
['my', 'stomach', 'is', 'killing', 'me', ':(']
['as', 'if', 'not', 'an', 'apology', "isn't", 'available', 'in', 'australia', ':(', '(']
['ksoo', 'u', 'dumb', 'butt', ':(']
['omg', 'why', 'am', 'i', 'getting', 'hate', 'for', 'being', 'a', 'sex', 'offender', ':(', 'poor', 'me', ':(', "i'm", 'a',

['im', 'so', 'irking', ':(']
['someone', 'say', 'thank', 'u', 'and', 'goodbye', 'to', 'chris', 'for', 'me', 'tomorrow', 'pls', ':(']
['i', 'need', 'nate', 'ruess', "'", 'album', ':(']
['i', 'could', 'just', 'really', 'use', 'a', 'hug', 'and', 'maybe', 'some', 'icecream', ':(']
['there', 'are', 'a', 'lot', 'of', 'good', 'comics', 'events', 'but', 'they', 'all', 'seem', 'to', 'be', 'for', 'kids', ':(', '[', 'part', 'of', 'the', 'schools', 'program', ']']
['looks', 'more', 'like', 'dog', 'food', '...', ':(', 'ugh', '!']
['i', 'miss', 'you', 'so', 'bad', ':(', '(', '(']
['aigoo', 'our', 'baby', ':(']
['want', 'to', 'watch', 'paper', 'town', ':-(', '(', '(']
['i', 'just', 'wanna', 'recover', ':(', '(']
[':-(', 'poor', 'boy']
['u', 'guys', 'are', 'together', '?', '?', ':(']
['i', 'saw', 'one', 'doujin', 'i', 'think', 'of', 'killua', 'getting', 'fucked', 'by', 'ging', 'i', 'was', 'a', 'bit', 'sad', 'cause', 'it', 'didnt', 'show', 'dicks', 'and', 'i', 'was', 'hoping', 'for', 'ginggon', ':(']
[

['i', 'always', 'get', 'like', '5', '4', '-', 'piece', 'spicy', 'nuggets', ':(']
['im', 'sooo', 'hungry', 'though', ':(']
['thigh', 'cramps', ':-(']
['to', 'my', 'fellow', 'morning', 'owls', ',', 'surely', 'you', 'have', 'noticed', 'the', 'sun', 'already', 'rising', 'a', 'bit', 'later', '...', 'sign', 'that', 'winter', 'is', 'coming', ':(']
['nowhere', 'near', 'being', 'tired', ':(']
['too', 'hot', ':(']
['dude', ':(', "i'm", 'scared', 'lol']
['raining', 'the', 'whole', 'day', 'in', 'mumbai', '.', 'don', '’', 't', 'feel', 'like', 'working', 'at', 'all', '.', ':(']
['best', 'get', 'me', 'and', 'my', 'little', 'lady', 'out', 'of', 'our', "pj's", 'need', 'to', 'go', 'wallpaper', 'for', 'me', 'mom', ':(', 'really', 'cba', 'today']
['yes', 'pls', 'far', 'i', 'need', 'it', ':(']
["didn't", 'even', 'know', 'it', 'could', 'get', 'hotter', 'than', 'this', 'fml', ':(']
['aw', 'bless', 'you', ',', 'this', 'made', 'me', 'smile', '!', "i'm", 'missing', 'you', 'too', ':(']
['hi', ',', 'life', ':(']


In [162]:
print(accuracy)

0.994


In [163]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    p = naive_bayes_predict(tweet, logprior, loglikelyhood)
    print(f'{tweet} -> {p:.2f}')

['i', 'am', 'happy']
I am happy -> 2.15
['i', 'am', 'bad']
I am bad -> -1.29
['this', 'movie', 'should', 'have', 'been', 'great', '.']
this movie should have been great. -> 2.14
['great']
great -> 2.14
['great', 'great']
great great -> 4.28
['great', 'great', 'great']
great great great -> 6.41
['great', 'great', 'great', 'great']
great great great great -> 8.55


In [165]:
my_tweet = 'you are bad :('
naive_bayes_predict(my_tweet, logprior, loglikelyhood)

['you', 'are', 'bad', ':(']


-8.802285344803796

In [169]:
naive_bayes_predict('The movie is very good and thrilling', logprior, loglikelyhood)

['the', 'movie', 'is', 'very', 'good', 'and', 'thrilling']


1.940532638693389

# Part 4: Filter words by Ratio of positive to negative counts

- Some words have more positive counts than others, and can be considered "more positive".  Likewise, some words can be considered more negative than others.
- One way for us to define the level of positiveness or negativeness, without calculating the log likelihood, is to compare the positive to negative frequency of the word.
    - Note that we can also use the log likelihood calculations to compare relative positivity or negativity of words.
- We can calculate the ratio of positive to negative frequencies of a word.
- Once we're able to calculate these ratios, we can also filter a subset of words that have a minimum ratio of positivity / negativity or higher.
- Similarly, we can also filter a subset of words that have a maximum ratio of positivity / negativity or lower (words that are at least as negative, or even more negative than a given threshold).

#### Implement `get_ratio()`
- Given the `freqs` dictionary of words and a particular word, use `lookup(freqs,word,1)` to get the positive count of the word.
- Similarly, use the `lookup()` function to get the negative count of that word.
- Calculate the ratio of positive divided by negative counts

$$ ratio = \frac{\text{pos_words} + 1}{\text{neg_words} + 1} $$

Where pos_words and neg_words correspond to the frequency of the words in their respective classes. 
<table>
    <tr>
        <td>
            <b>Words</b>
        </td>
        <td>
        Positive word count
        </td>
         <td>
        Negative Word Count
        </td>
  </tr>
    <tr>
        <td>
        glad
        </td>
         <td>
        41
        </td>
    <td>
        2
        </td>
  </tr>
    <tr>
        <td>
        arriv
        </td>
         <td>
        57
        </td>
    <td>
        4
        </td>
  </tr>
    <tr>
        <td>
        :(
        </td>
         <td>
        1
        </td>
    <td>
        3663
        </td>
  </tr>
    <tr>
        <td>
        :-(
        </td>
         <td>
        0
        </td>
    <td>
        378
        </td>
  </tr>
</table>

In [170]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words
        word: string to lookup

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 1, 'ratio': 0.0}
    
    pos_neg_ratio['positive'] = lookup(freqs, word, 1)
    pos_neg_ratio['negative'] = lookup(freqs, word, 0)
    
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1)/(pos_neg_ratio['negative'] + 1)
    return pos_neg_ratio

In [171]:
get_ratio(freqs, 'happi')

{'positive': 161, 'negative': 18, 'ratio': 8.526315789473685}

#### Implement `get_words_by_threshold(freqs,label,threshold)`

* If we set the label to 1, then we'll look for all words whose threshold of positive/negative is at least as high as that threshold, or higher.
* If we set the label to 0, then we'll look for all words whose threshold of positive/negative is at most as low as the given threshold, or lower.
* Use the `get_ratio()` function to get a dictionary containing the positive count, negative count, and the ratio of positive to negative counts.
* Append a dictionary to a list, where the key is the word, and the dictionary is the dictionary `pos_neg_ratio` that is returned by the `get_ratio()` function.
An example key-value pair would have this structure:
```
{'happi':
    {'positive': 10, 'negative': 20, 'ratio': 0.5}
}
```

In [175]:
def get_words_by_threshold(freqs, label, threshold):
    '''
    Input:
        freqs: dictionary of words
        label: 1 for positive, 0 for negative
        threshold: ratio that will be used as the cutoff for including a word in the returned dictionary
    Output:
        word_set: dictionary containing the word and information on its positive count, negative count, and ratio of positive to negative counts.
        example of a key value pair:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}
    
    for key in freqs.keys():
        word, _ = key
        
        # get the positive/negative ratio for a word
        pos_neg_ratio = get_ratio(freqs, word)
        
        # if the label is 1  and the ratio is greater than or equal to the threshold....
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:
            word_list[word] = pos_neg_ratio
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
            word_list[word] = pos_neg_ratio
            
    return word_list
    
    
    
    

In [176]:
# Test your function: find negative words at or below a threshold
get_words_by_threshold(freqs, label=0, threshold=0.05)

{':(': {'positive': 1, 'negative': 3663, 'ratio': 0.0005458515283842794},
 ':-(': {'positive': 0, 'negative': 378, 'ratio': 0.002638522427440633},
 'zayniscomingbackonjuli': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '26': {'positive': 0, 'negative': 20, 'ratio': 0.047619047619047616},
 '>:(': {'positive': 0, 'negative': 43, 'ratio': 0.022727272727272728},
 'lost': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '♛': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 '》': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 'beli̇ev': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'wi̇ll': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'justi̇n': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｓｅｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｍｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776}}

In [177]:
# Test your function; find positive words at or above a threshold
get_words_by_threshold(freqs, label=1, threshold=10)

{'followfriday': {'positive': 23, 'negative': 0, 'ratio': 24.0},
 'commun': {'positive': 27, 'negative': 1, 'ratio': 14.0},
 ':)': {'positive': 2847, 'negative': 2, 'ratio': 949.3333333333334},
 'flipkartfashionfriday': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':d': {'positive': 498, 'negative': 0, 'ratio': 499.0},
 ':p': {'positive': 104, 'negative': 0, 'ratio': 105.0},
 'influenc': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':-)': {'positive': 543, 'negative': 0, 'ratio': 544.0},
 "here'": {'positive': 20, 'negative': 0, 'ratio': 21.0},
 'youth': {'positive': 14, 'negative': 0, 'ratio': 15.0},
 'bam': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'warsaw': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'shout': {'positive': 11, 'negative': 0, 'ratio': 12.0},
 ';)': {'positive': 22, 'negative': 0, 'ratio': 23.0},
 'stat': {'positive': 51, 'negative': 0, 'ratio': 52.0},
 'arriv': {'positive': 57, 'negative': 4, 'ratio': 11.6},
 'via': {'positive': 60, 'negative': 1, 

# Part 5: Error Analysis

In this part you will see some tweets that your model missclassified. Why do you think the misclassifications happened? Were there any assumptions made by the naive bayes model?

In [183]:
# Some error analysis done for you
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelyhood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            process_tweet(x)).encode('ascii', 'ignore')))

Truth Predicted Tweet
['bro', ':', 'u', 'wan', 'cut', 'hair', 'anot', ',', 'ur', 'hair', 'long', 'liao', 'bo', 'me', ':', 'since', 'ord', 'liao', ',', 'take', 'it', 'easy', 'lor', 'treat', 'as', 'save', 'leave', 'it', 'longer', ':)', 'bro', ':', 'lol', 'sibei', 'xialan']
['is', 'back', '!', 'thnx', 'god', '!', '!', '!', "i'm", 'so', 'happy', ':)']
['thought', 'it', 'was', 'my', 'ears', 'which', 'were', 'malfunctioning', ',', 'thank', 'goodness', 'you', 'cleared', 'that', 'one', 'up', 'with', 'an', 'apology', ':-)']
["'", 'stuck', 'in', 'the', 'centre', 'right', 'with', 'you', '.', 'clowns', 'to', 'the', 'right', ',', 'jokers', 'to', 'the', 'left', '...', "'", ':)']
['happy', 'friday', ':-)']
['we', 'are', 'following', 'you', 'now', ':)', 'x']
['my', 'teenchoice', 'for', 'choiceinternationalartist', 'is', 'superjunior', 'fighting', 'oppa', ':D']
["it's", 'my', 'birthday', 'today', 'so', 'for', 'my', 'birthday', 'wish', 'i', 'hope', "there's", 'good', 'news', 'about', 'ben', 'soon', ':-)

['and', 'i', 'have', 'it', ':)']
['boom', '.', "it's", 'a', 'date', '!', ':D', '"', 'z']
['followed', 'already', '!', 'thanks', '!', ':)']
['thank', 'you', '!', ':)']
['will', 'you', 'be', 'my', 'happy', 'ending', '?', 'insyaallah', ':)']
['thats', 'the', 'way', 'we', 'see', 'it', '.', 'west', 'ham', 'are', 'a', 'shit', 'small', 'club', 'in', 'london', 'and', 'villa', 'are', 'the', '3rd', 'biggest', 'in', 'brum', ':)']
['denis', 'has', 'a', 'vagina', '?', 'rly', '?', ':D', 'poor', 'thing', ':D']
['smile', '.', 'its', 'sunnah', ':)', 'غردلي', 'عن']
['hi', '!', 'i', 'see', 'u', 'like', 'fourfiveseconds', 'and', 'think', 'u', 'might', 'like', '"', 'deaf', 'ears', '"']
['share', 'it', 'when', 'ready', '!', ':)']
['oh', 'my', ',', 'the', 'white', 'rabbit', 'is', 'the', 'cutest', 'thing', "i've", 'ever', 'seen', ',', 'no', 'wait', "it's", 'all', 'adorable', ':)']
["you're", 'welcome', ':)', 'have', 'a', 'lovely', 'weekend', 'with', 'izzy', '.']
['happy', 'birthday', 'mitch', ':-)', 'i', 'wis

['oh', 'gina', ',', 'you', 'have', 'been', 'busy', ':-)', "can't", 'wait', 'to', 'see', 'you', 'and', 'hear', 'all', 'xxx', 'soproud']
['happy', 'friday', 'everyone', '!', 'we', 'hope', "you've", 'all', 'had', 'a', 'fantastic', 'week', '!', ':)', 'friday']
['yes', 'the', 'one', 'and', 'only', 'team', ':)']
['happy', 'belated', 'birthday', 'sweetie', ':)']
['morning', 'emma', ':)']
['thanks', 'guys', ':)']
['then', 'why', 'would', 'he', 'say', 'they', 'did', '?', ':-)']
['love', 'makes', 'two', 'hearts', 'one', '.', ':)']
['thanks', 'for', 'adding', 'us', 'to', 'your', 'list', '!', ':)', 'make', 'sure', 'to', 'keep', 'in', 'touch', 'for', 'more', 'news', 'of', 'our', 'light', 'bulbs', 'made', 'from', 'corn', 'waste', '.']
['thanks', 'mrs', 'bee', '!', '!', 'xxx', ':-)']
['thankyou', ':)']
['close', 'enought', '?', ':)']
['good', 'luck', '...', 'another', 'potential', 'favourite', 'watering', 'hole', ':-)']
['bad', 'boys', ':)', 'burgers', 'melbourneburgers']
['who', 'wants', 'to', 'ft',

[':(', 'but', 'wtf', 'am', 'i', 'supposed', 'to', 'do', 'now', 'without', 'her']
['headache', 'strike', ':(']
['english', 'weather', 'needs', 'to', 'fix', 'up', ':(', '(']
['live', 'fam', 'bam', 'but', 'i', 'have', 'a', 'cough', ':(']
['absolutely', 'gutted', 'all', 'the', 'james', 'bay', 'tickets', 'have', 'sold', 'out', 'for', 'manchester', ':(']
['seventh', 'spot', 'na', 'lang', ':(', 'otwolgrandtrailer']
['splendour', ':(']
['omg', '!', 'swedish', 'hair', 'metal', 'legends', 'häirførce', 'in', 'the', 'studio', 'while', 'i', 'am', 'on', 'leave', '?', '?', '?', 'nooo', ':(', 'givecodpieceachance']
["it's", 'sore', ',', 'alice', '.', ':(']
["i'm", 'not', 'ready', 'to', 'work', 'yet', ':(']
['someone', 'unfaved', ':(', '(']
['stiles', ':(']
['funny', 'mo', ':(']
['someone', 'explain', 'sandrabland', ':(']
['dont', 'be', 'sad', ':(', '(', '(', 'ily']
['no', ',', 'i', 'am', 'going', 'to', 'spend', 'the', 'night', 'in', 'prague', 'and', 'then', 'leaving', 'tomorrow', ':(']
['one', 'of', '

['still', 'sad', 'that', 'we', "haven't", 'fixed', 'my', 'cars', 'window', 'bc', 'i', "can't", 'drive', 'anywhere', 'without', 'being', 'scared', "it'll", 'shatter', 'on', 'me', ':(']
['i', 'only', 'have', 'the', 'morning', 'off', 'work', 'unfortunately', ':(', 'are', 'you', 'at', 'gdce', '/', 'gamescom', '?']
['hay', 'idk', 'baby', ':(', '😭', '😭', '😭']
['oh', 'no', ':(', "do't", 'think', 'that', '.', 'do', 'you', 'have', 'a', 'parcel', 'coming', 'from', 'yodel', 'now', '?', 'if', 'so', 'hit', 'follow', '&', 'dm', 'over', 'your', 'tracking', 'num', '.', 'chelsea']
['i', 'wish', 'i', 'could', 'be', 'friends', 'with', 'everybody', ':(', 'lmfaooo']
['oooouch', '!', '!', '!', 'my', 'poor', 'pinky', 'toe', '👣', 'good', 'thing', 'i', 'work', 'for', 'podiatrists', '.', 'just', 'waiting', 'until', 'morning', 'so', 'i', 'can', 'go', 'in', '.', 'hurry', 'and', 'be', 'morning', ':(']
['morning', ',', 'i', 'miss', 'you', ':(', ':', '*']
['i', 'miss', 'my', 'old', 'house', 'because', 'you', 'could'

['lol', 'we', 'did', 'it', 'at', '2am', 'last', 'night', '.', 'my', 'face', 'though', '?', 'people', 'think', "i'm", 'perpetually', 'pissed', 'off', 'or', 'sad', 'b', '/', 'c', 'of', 'my', 'resting', 'bitchface', '.', ':(']
[':-(', '@', 'missing', 'all', 'the', 'fun']
[':-(', 'please', 'notice', 'me', 'z']
['only', 'a', 'few', 'people', 'have', 'wished', 'my', 'birthday', '.', ':(']
[':(', 'the', 'world', 'is', 'awful', '.']
['i', 'let', 'you', 'down', 'and', 'now', 'i', 'feel', 'like', 'complete', 'shit', '.', ':(']
['uh', 'huh', ':(', 'two', 'times', ',', 'because', "i'm", 'super', 'clever', 'like', 'that']
[':(', 'bitch', 'you', 'really', 'went', '?']
['when', 'fave', 'unfollows', ':(', '😩', '💖']
['i', 'really', 'want', 'to', 'visit', 'iceland', ':(']
['zayn_come_back_we_miss_you', '<3', '<3', ':(', 'so', 'much']
['starving', ':-(']
['bc', 'i', 'am', 'feeling', 'very', 'creepy', 'today', ':', ')', '/', '/', 'pmsl', '.', 'mianhe', ',', 'milkeu', ':(']
['i', 'wanna', 'meet', 'the', 'b

# Part 6: Predict with your own tweet

In this part you can predict the sentiment of your own tweet.

In [185]:
# Test with your own tweet - feel free to modify `my_tweet`
my_tweet = 'I am happy because I am learning :)'

p = naive_bayes_predict(my_tweet, logprior, loglikelyhood)
print(p)

['i', 'am', 'happy', 'because', 'i', 'am', 'learning', ':)']
9.573774904705935
